In [1]:
import os
import mlflow
import numpy as np
import pandas as pd

from math import sqrt
from joblib import delayed
from joblib import Parallel

from warnings import catch_warnings
from warnings import filterwarnings
from multiprocessing import cpu_count
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from pathlib import Path
from functools import reduce
from datetime import datetime

from prophet import Prophet

Importing plotly failed. Interactive plots will not work.


# Data

In [65]:
df = pd.read_csv('./../final_dfs/for_modeling/df_final_date_wide_2007.csv', converters={'ags2': str, 'ags5': str} )
df.set_index('ags5', drop=True, inplace=True)
df = df.drop("Unnamed: 0", axis=1)
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df = df[(df['date'] >= '2010-01-01') & (df['date'] < '2020-01-01')]
df = df.rename(columns={'date': 'ds', 'unemployment_rate': 'y'})
df

,ags2,date,number_of_company_deletions,number_of_company_liquidations,number_of_start_ups,number_of_companies_administration,number_of_companies_agriculture,number_of_companies_arts_entertainment,number_of_companies_communication,number_of_companies_construction,...,realized_short_time_work_companies,realized_short_time_work_people,underemployment_without_short_time _work,unemployment_benefit_entitled,unemployment_benefit_recipients,unemployment_rate,registerd_jobs,unemployed,employees_social_security_at_work,employees_social_security_at_residence
ags5,,,,,,,,,,,,,,,,,,,,,
1001,1,2007-05-01,4.0,2.0,21.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,12.7,784,5559,38319,24519
1001,1,2007-06-01,8.0,6.0,18.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,12.2,932,5323,38266,24560
1001,1,2007-07-01,9.0,0.0,18.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,12.5,862,5454,38237,24561
1001,1,2007-08-01,6.0,4.0,10.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,12.3,819,5377,38833,24936
1001,1,2007-09-01,11.0,4.0,6.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,11.7,725,5132,38840,25072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16077,16,2020-07-01,2.0,1.0,5.0,114.0,100.0,122.0,44.0,457.0,...,311.0,2709.0,4505.0,6661.0,1022.0,7.2,664,3242,27356,33148
16077,16,2020-08-01,1.0,1.0,4.0,114.0,100.0,121.0,44.0,457.0,...,264.0,2104.0,4432.0,6653.0,1024.0,7.3,692,3271,27490,33287
16077,16,2020-09-01,1.0,0.0,5.0,114.0,100.0,121.0,44.0,455.0,...,224.0,1750.0,4277.0,6473.0,970.0,7.0,806,3129,27765,33583


In [72]:
df[['date', 'number_of_company_deletions', 'number_of_company_liquidations', 'number_of_start_ups',
       'unemployment_rate', 'registerd_jobs',
       'employees_social_security_at_work',
       'employees_social_security_at_residence']]

Index(['ags2', 'date', 'number_of_company_deletions',
       'number_of_company_liquidations', 'number_of_start_ups',
       'number_of_companies_administration', 'number_of_companies_agriculture',
       'number_of_companies_arts_entertainment',
       'number_of_companies_communication', 'number_of_companies_construction',
       'number_of_companies_domestic_staff',
       'number_of_companies_economic_services',
       'number_of_companies_education', 'number_of_companies_energy',
       'number_of_companies_extraterritorial',
       'number_of_companies_financial_and_insurance',
       'number_of_companies_health_and_social_services',
       'number_of_companies_hospitality', 'number_of_companies_manufacturing',
       'number_of_companies_mining', 'number_of_companies_real_estat',
       'number_of_companies_rendering_other_services',
       'number_of_companies_repair_motor_vehicles',
       'number_of_companies_technical_services',
       'number_of_companies_transport', 'numbe

# 401 Model for each kreis

In [63]:
# one-step forecast
def Prophet_forecast(history, config):
    g, sps, ys, sm = config
    # define model
    model = Prophet(growth = g, seasonality_prior_scale = sps, yearly_seasonality = ys, seasonality_mode = sm)
    # fit model
    model_fit = model.fit(history)
    future = model_fit.make_future_dataframe(periods=1)
    # make one step forecast
    res = model_fit.predict(future)
    res.to_csv(f"runs_res/{g}_{sps}_{ys}_{sm}")
    return res.iloc[-13:-1][["ds","yhat"]]

In [50]:
config = ['linear', 15.0, True, 'additive']
Prophet_forecast(df, config)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


SystemError: <class 'stanfit4anon_model_dfdaf2b8ece8a02eb11f050ec701c0ec_3159786415757848982.PyStanHolder'> returned a result with an error set

In [55]:
# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

In [56]:
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

In [57]:
(train_test_split(df, 12)[1] )

,ags2,ds,number_of_company_deletions,number_of_company_liquidations,number_of_start_ups,number_of_companies_administration,number_of_companies_agriculture,number_of_companies_arts_entertainment,number_of_companies_communication,number_of_companies_construction,...,realized_short_time_work_companies,realized_short_time_work_people,underemployment_without_short_time _work,unemployment_benefit_entitled,unemployment_benefit_recipients,y,registerd_jobs,unemployed,employees_social_security_at_work,employees_social_security_at_residence
ags5,,,,,,,,,,,,,,,,,,,,,
16077,16,2019-01-01,7.0,4.0,5.0,93.0,106.0,97.0,38.0,486.0,...,NaN,NaN,4836.0,7762.0,1012.0,8.0,997,3675,27701,33710
16077,16,2019-02-01,4.0,1.0,4.0,-99.0,-99.0,-99.0,-99.0,-99.0,...,NaN,NaN,4842.0,7711.0,1005.0,8.0,1014,3673,27702,33723
16077,16,2019-03-01,2.0,1.0,2.0,101.0,84.0,77.0,34.0,425.0,...,NaN,NaN,4740.0,7685.0,927.0,7.6,1048,3489,27818,33784
16077,16,2019-04-01,3.0,0.0,1.0,111.0,95.0,111.0,39.0,423.0,...,NaN,NaN,4615.0,7602.0,822.0,7.2,1107,3319,27992,33885
16077,16,2019-05-01,7.0,0.0,2.0,112.0,101.0,123.0,44.0,482.0,...,NaN,NaN,4575.0,7495.0,763.0,7.1,1059,3214,27960,33890
16077,16,2019-06-01,5.0,0.0,1.0,111.0,95.0,120.0,42.0,475.0,...,NaN,NaN,4497.0,7387.0,767.0,7.0,1047,3140,27912,33883
16077,16,2019-07-01,1.0,3.0,4.0,111.0,96.0,120.0,42.0,475.0,...,NaN,NaN,4504.0,7294.0,821.0,7.0,1044,3146,27829,33779
16077,16,2019-08-01,3.0,3.0,2.0,111.0,96.0,121.0,42.0,477.0,...,NaN,NaN,4513.0,7188.0,842.0,7.0,990,3168,27980,33983
16077,16,2019-09-01,1.0,1.0,3.0,111.0,93.0,116.0,38.0,451.0,...,NaN,NaN,4390.0,7145.0,786.0,6.5,984,2929,28228,34292


In [58]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
    # split dataset
    train, test = train_test_split(data, n_test)
    # fit model and make forecast for history
    yhat = Prophet_forecast(train, cfg)
    # estimate prediction error
    error = measure_rmse(test['y'], yhat['yhat'])
    return error

In [59]:
def run_model(data, n_test): 
        for g in ['linear']: 
            for sps in [10.0, 15.0]: 
                for ys in ['auto', True]: 
                    for sm in ['additive', 'multiplicative']:
                        with mlflow.start_run():
                            params = [g, sps, ys, sm]

                            ags5 = list(set(df.index.values.tolist()))
                            rmse = []
                            for i in ags5:
                                df_ags5 = df.filter(regex=i, axis=0)
                                current_rmse = walk_forward_validation(df_ags5, n_test, params)
                                print(current_rmse)
                                rmse.append(current_rmse)

                            # Log params
                            params = {"growth":g, "sps":sps, "ys":ys, "sm":sm}

                            mlflow.log_params(params)

                            # Log metrics
                            mlflow.log_metric('rmse', np.mean(rmse))

In [64]:
run_model(df, 12)

<ipython-input-63-c7ee9dab0c26>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history['cap'] = 15
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
<ipython-input-63-c7ee9dab0c26>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history['cap'] = 15
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:p

0.2114899910295795


<ipython-input-63-c7ee9dab0c26>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history['cap'] = 15
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.4462786197273205


<ipython-input-63-c7ee9dab0c26>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history['cap'] = 15
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

0.5700643472764643


<ipython-input-63-c7ee9dab0c26>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history['cap'] = 15
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.44270358567252704


<ipython-input-63-c7ee9dab0c26>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history['cap'] = 15
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.40782904551861376
0.41243348037242716


In [20]:
np.mean([10.0, 15.0])

12.5

In [9]:
# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
    result = None
    # convert config to a key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
        result = walk_forward_validation(data, n_test, cfg)
    else:
            # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")
                result = walk_forward_validation(data, n_test, cfg)
        except:
            error = None
    # check for an interesting result
#     if result is not None:
#         print(' > Model[%s] %.3f' % (key, result))
        
    print('.', end='')
    return (key, result)

----------
!mlflow experiments create --experiment-name 401ModelsTemp

Created experiment '401ModelsTemp' with id 6

---------

In [ ]:
# data split
n_test = 12
# model config
cfg = [('linear'), (10.0)]
# train 401 models
print("Training", end=' ')
failed_ags5 = []
rmse_scores = []

with mlflow.start_run(experiment_id=6):
    
    for ags5, df_for_kreis in df_y_grouped:
        
        data = list(df_for_kreis.sort_values('ds').drop(['ds'], axis=1).copy()['y'])
        
        try: 
        
            score = score_model(data, n_test, cfg)

            if score is None:
                raise Exception("No Score")
            
            rmse_scores.append(score[1])
            print('.', end='')
        except:
            failed_ags5.append(ags5)
            print('!', end='')
            
    params = {}
    params['p'], params['d'], params['q'] = cfg[0]
    params['s_p'], params['s_d'], params['s_q'], params['m'] = cfg[1]
    params['t'] = cfg[2]
    
    net_rmse = np.mean([rmse**2 for rmse in rmse_scores])**(0.5)
    
    
    mlflow.log_params(params)
    mlflow.log_metric('net_rmse', net_rmse)
      
    

print('\ndone')
print("Failed ags5:", failed_ags5)

Training ........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [ ]:
rmse_scores

In [ ]:
params

In [37]:
m = Prophet()
m.fit(df)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [38]:
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
480,2020-11-26
481,2020-11-27
482,2020-11-28
483,2020-11-29
484,2020-11-30


In [42]:
Prophet?